# day 13

https://adventofcode.com/2020/day/13

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day13.txt')

LOGGER = logging.getLogger('day13')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """939
7,13,x,x,59,x,31,19"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
def parse(s):
    ts, bus_ids = s.split('\n')
    ts = int(ts)
    bus_ids = [int(_) for _ in bus_ids.split(',') if _ != 'x']
    return ts, bus_ids

In [ ]:
parse(test_data)

In [ ]:
parse(load_data())

#### function def

In [ ]:
def q_1(data):
    ts, bus_ids = parse(data)
    best_id = min(bus_ids, key=lambda k: k - (ts % k))
    return best_id * (best_id - (ts % best_id))

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 295
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
import math

def lcm(a, b):
    return abs(a * b) // math.gcd(a, b)

In [ ]:
def parse(s):
    _, bus_ids = s.split('\n')
    return [[i, int(b)] for (i, b) in enumerate(bus_ids.split(','))
            if b != 'x']

In [ ]:
parse(load_data())

In [ ]:
def q_2(data):
    reqs = parse(data)
    
    t = 0
    step = reqs[0][1]
    goodness = 1
    while goodness < len(reqs):
        offset, period = reqs[goodness]
        if (t + offset) % period == 0:
            step = lcm(step, period)
            goodness = goodness + 1
        else:
            t += step

    return t

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 1068781
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin

# looking at wikipedia

methods [here](https://en.wikipedia.org/wiki/Chinese_remainder_theorem)

In [ ]:
parse(test_data)

above is equivalent to

$$
t = 0 \mod 7 \\
t = 1 \mod 13 \\
t = 4 \mod 59 \\
t = 6 \mod 31 \\
t = 7 \mod 19 \\
$$

consider each of the above a statement of the form $t = a_i \mod n_i$

the Bezout's identity for the first two equations suggests

$$
\exists m_0, m_1: 7 m_0 + 13 m_1 = 1
$$

and wikipedia suggest using the [extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) for that

In [ ]:
def extended_gcd(a, b):
    r, r_prev = a, b
    s, s_prev = 0, 1
    t, t_prev = 1, 0
    
    while r != 0:
        quo = r_prev // r
        r_prev, r = r, r_prev - quo * r
        s_prev, s = s, s_prev - quo * s
        t_prev, t = t, t_prev - quo * t
        LOGGER.debug(r)
    
    return {'bezout': (s_prev, t_prev),
            'gcd': r_prev,
            'quotients': (t, s)}

In [ ]:
# the wikipedia example
#   x = 0 mod 3
#   x = 3 mod 4
#   x = 4 mod 5
a0, n0 = 0, 3
a1, n1 = 3, 4
m1, m0 = extended_gcd(n0, n1)['bezout']
assert m0 * n0 + m1 * n1 == 1
t = (a0 * m1 * n1
     + a1 * m0 * n0)
if t < 0:
    t %= n0 * n1
t

In [ ]:
# these are treated now as the first of a pair
#   x = 3 mod 12
#   x = 4 mod 5
a0, n0 = 3, 12
a1, n1 = 4, 5
m1, m0 = extended_gcd(n0, n1)['bezout']
assert m0 * n0 + m1 * n1 == 1
t = (a0 * m1 * n1
     + a1 * m0 * n0)
if t < 0:
    t %= n0 * n1
t

In [ ]:
for (a, n) in [(0, 3), (3, 4), (4, 5)]:
    assert 39 % n == a

okay so given the above, the new function is straight forward:

In [ ]:
def step(a0, n0, a1, n1):  
    m1, m0 = extended_gcd(n0, n1)['bezout']
    t = (a0 * m1 * n1
         + a1 * m0 * n0)
    if t < 0:
        t %= n0 * n1
    return t, n0 * n1

In [ ]:
def q_2(data):
    reqs = parse(data)
    
    a0, n0 = reqs.pop()
    while reqs:
        a1, n1 = reqs.pop()
        LOGGER.debug(f"a0, n0, a1, n1 = {a0, n0, a1, n1}")
        a0, n0 = step(a0, n0, a1, n1)
    return a0

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 1068781
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

In [ ]:
q_2(test_data)

In [ ]:
parse(test_data)